In [1]:
import os
from typing import Union
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras_core as K
from keras_core import layers
from keras_core import losses
from keras_core import metrics
from keras_core import optimizers
from keras_core import initializers
import keras_nlp
from dataclasses import dataclass

2023-09-18 12:21:03.914451: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


In [2]:
data_dir = "../data"
train_path = os.path.join(data_dir, "shakespeare/train.bin")
val_path = os.path.join(data_dir, "shakespeare/val.bin")

In [38]:
@dataclass
class GPTConfig:
    # GPT configs
    block_size: int = 64 # 1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 4 # 12
    n_head: int = 12
    n_embd: int = 256 # 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster
    layer_norm_epsilon: float = 1e-05

    # Train configs
    batch_size = 16
    weight_decay = 1e-01
    beta1 = 0.9
    beta2 = 0.95
    decay_lr = True # whether to decay the learning rate
    warmup_iters = 100 # 2000 # how many steps to warm up for

config = GPTConfig()

In [39]:
class Block(layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.ln1 = layers.LayerNormalization(epsilon=config.layer_norm_epsilon) # TODO: bias?
        self.ln2 = layers.LayerNormalization(epsilon=config.layer_norm_epsilon)
        mha = layers.MultiHeadAttention(num_heads=config.n_head, key_dim=config.n_embd // config.n_head)
        self.attn = lambda x, training: mha(x, x, training=training, use_causal_mask=True)
        self.mlp = K.Sequential([
            layers.Dense(
                units=4*config.n_embd, use_bias=True, activation="gelu",
                kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.02),
                bias_initializer=initializers.Zeros(),
            ),
            layers.Dense(
                units=config.n_embd, use_bias=True,
                kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.02),
                bias_initializer=initializers.Zeros(),
            ),
            layers.Dropout(config.dropout)
        ])

    def call(self, x, training=None):
        x = x + self.attn(self.ln1(x), training=training)
        x = x + self.mlp(self.ln2(x), training=training)
        return x
    
class GPT(K.Model):
    def __init__(self, config: GPTConfig, **kwargs):
        super().__init__(name="coreGPT", **kwargs)
        self.config = config

        # input embedding
        self.emb = keras_nlp.layers.TokenAndPositionEmbedding(
            vocabulary_size=config.vocab_size,
            sequence_length=config.block_size,
            embedding_dim=config.n_embd,
            embeddings_initializer=initializers.RandomNormal(mean=0.0, stddev=0.02)
        )
        self.drop = layers.Dropout(config.dropout)
        # transformer blocks
        self.blocks = [Block(config) for _ in range(config.n_layer)]
        # decoder head
        self.ln_f = layers.LayerNormalization(axis=-1)
        self.head = layers.Dense(
            units=config.vocab_size, use_bias=False,
            kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.02),
        )

    def call(self, inputs, training=None):
        B, T = inputs.shape
        # embed sentence
        x = self.emb(inputs)
        x = self.drop(x, training=training)
        # attention
        for block in self.blocks:
            x = block(x, training=training)
        # compute logits
        x = self.ln_f(x)
        x = self.head(x)
        return x
    
    def summary(self):
        x = K.Input(shape=[self.config.block_size], batch_size=config.batch_size, dtype="int32")
        dummy = K.Model(inputs=x, outputs=self.call(x), name=self.name)
        return dummy.summary()

In [40]:
inputs = K.Input(shape=[config.block_size], dtype="int32")

model = GPT(config)
model.compile(
    optimizer=optimizers.AdamW(learning_rate=6e-4, weight_decay=config.weight_decay),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy(name='accuracy')],
)
model.summary()

Model: "coreGPT"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (16, 64)                  │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ token_and_position_embedding_2  │ (16, 64, 256)             │ 12,894,208 │
│ (TokenAndPositionEmbedding)     │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_18 (Dropout)            │ (16, 64, 256)             │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ block_8 (Block)                 │ (16, 64, 256)             │    526,592 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ block_9 (Block)                 │ (16, 64, 256)             │    526,592 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ block_10 (Block)                │ (16, 64, 256)             │    526,592 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ block_11 (Block)                │ (16, 64, 256)             │    526,592 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_normalization_26          │ (16, 64, 256)             │        512 │
│ (LayerNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_26 (Dense)                │ (16, 64, 50304)           │ 12,877,824 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 27,878,912 (106.35 MB)

 Trainable params: 27,878,912 (106.35 MB)

 Non-trainable params: 0 (0.00 B)

In [85]:
train_data = np.memmap(train_path, dtype=np.uint16, mode='r')
val_data = np.memmap(val_path, dtype=np.uint16, mode='r')

n_batch_train = (len(train_data)-config.block_size)//config.batch_size
n_batch_val = (len(val_data)-config.block_size)//config.batch_size

def get_windowed_tf_dataset(data: Union[np.memmap, np.array]):
    x = (
        tf.data.Dataset.from_tensor_slices(data[:-1])
        .window(config.block_size, shift=1, stride=1, drop_remainder=True)
        .flat_map(lambda x: x.batch(config.block_size))
    )
    y = (
        tf.data.Dataset.from_tensor_slices(data[1:])
        .window(config.block_size, shift=1, stride=1, drop_remainder=True)
        .flat_map(lambda x: x.batch(config.block_size))
    )

    return (
        tf.data.Dataset
        .zip((x, y))
        .batch(batch_size=config.batch_size,
               drop_remainder=True,
               num_parallel_calls=tf.data.AUTOTUNE)
        .repeat()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )

train_dataset = get_windowed_tf_dataset(train_data)
val_dataset = get_windowed_tf_dataset(val_data)

train_data, val_data = None, None

In [83]:
history = model.fit(
    train_dataset,
    steps_per_epoch=n_batch_train,
    epochs=2,
    validation_data=val_dataset,
    validation_steps=n_batch_val,
    verbose=1
)
    # batch_size=None,
    # epochs=1,
    # use_multiprocessing=True,
    # verbose="auto",
    # callbacks=None,
    # validation_split=0.0,
    # validation_data=None,
    # shuffle=True,
    # class_weight=None,
    # sample_weight=None,
    # initial_epoch=0,
    # steps_per_epoch=None,
    # validation_batch_size=None,
    # validation_freq=1,
    # max_queue_size=10,
    # workers=1,

Epoch 1/2


2023-09-18 14:11:06.257060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype uint16 and shape [316]
	 [[{{node Placeholder/_0}}]]
2023-09-18 14:11:06.257581: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype uint16 and shape [316]
	 [[{{node Placeholder/_8}}]]


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7014 - loss: 1.0106

2023-09-18 14:11:26.672404: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype uint16 and shape [142]
	 [[{{node Placeholder/_8}}]]
2023-09-18 14:11:26.672965: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype uint16 and shape [142]
	 [[{{node Placeholder/_0}}]]


15/15 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.7042 - loss: 1.0013 - val_accuracy: 0.0745 - val_loss: 9.3600
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.7630 - loss: 0.7862 - val_accuracy: 0.1067 - val_loss: 9.6398


In [84]:
history.history

{'accuracy': [0.7466796636581421, 0.799023449420929],
 'loss': [0.8615785241127014, 0.6976556181907654],
 'val_accuracy': [0.074462890625, 0.106689453125],
 'val_loss': [9.359951972961426, 9.639775276184082]}

In [15]:
inp = np.random.random_integers(0, 100, size=(1, config.n_seq,))
inp.shape

/var/folders/vg/cjz3vt2s5szcrgwhqp9hfz9w0000gn/T/ipykernel_12910/4236967733.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 100 + 1) instead
  inp = np.random.random_integers(0, 100, size=(1, config.n_seq,))


(1, 256)

In [17]:
out = model.predict(inp, batch_size=1)
out.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step


(1, 256, 50304)

In [18]:
out[0][0]

array([ 0.29266614, -0.02570427, -0.08666821, ...,  0.2424404 ,
       -0.1005659 ,  0.25224355], dtype=float32)

In [ ]:
for _ in range(2):
    

In [45]:
inp = np.array(
    [
        [
            [0.0, 0.0, 0.0, 0.0],
            [100, 200, 300, 400],
            [0.1, 0.3, 0.7, 0.9],
        ]
    ]
)

print(inp.shape)
inp

(1, 3, 4)


array([[[0.e+00, 0.e+00, 0.e+00, 0.e+00],
        [1.e+02, 2.e+02, 3.e+02, 4.e+02],
        [1.e-01, 3.e-01, 7.e-01, 9.e-01]]])

In [46]:
model.predict([inp], batch_size=3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


array([[[   0.      ,    0.      ,    0.      ,    0.      ],
        [ -51.026886, -188.48923 ,   14.233276, -163.07217 ],
        [ -50.939663, -188.4063  ,   14.111473, -163.03067 ]]],
      dtype=float32)

In [6]:
inputs = np.ones(shape=(1, 50), dtype="int32")